<a href="https://colab.research.google.com/github/raskutti/math-fun/blob/main/numbers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solving "Numbers"

This colab solves an easier version of the game from [Letters and Numbers](https://en.wikipedia.org/wiki/Letters_and_Numbers) (essentially the same as [Countdown](https://en.wikipedia.org/wiki/Countdown_(game_show)).

In [ ]:
import numpy as np

def _is_positive_integer(n):
  return n > 0 and isinstance(n, int)

# Brute force approach

For a given $m \in \mathbb{N}$ and $n_0, ..., n_{N-1} \in \mathbb{N}$, find a set of elements and mathematical operators (addition, subtraction, multiplication, division) that yields $m$.

- Elements from $\{n_0, ..., n_{N-1}\}$ can be used in any order **without replacement**
- Mathematical operators from $\{+, -, *, /\}$ can be used in order **with replacement**

In [ ]:
import operator
import itertools

# +, -, *, /
all_operations = [operator.add, operator.sub, operator.mul, operator.floordiv]

In [ ]:
def all_permutations(arr):
  # O(sum_1^n i!)
  """A generator for permutations of a list, in increasing length.

  The permutations are of any non-zero length, and order matters."""

  for n in range(1, len(arr) + 1):
    for c in itertools.combinations(arr, n):
      for p in itertools.permutations(c):
        yield p

In [ ]:
def check_permutation(elements, operations, answer):
  # O(n)
  """Checks if a permutation of elements and operations returns the answer."""
  n = len(elements) - 1
  guess = elements[0]

  for i in range(n):
    el, op = elements[i+1], operations[i]

    # If a solution could use a negative integer (e.g. 5 = 2 - 3 + 6) then it
    # can be accomplished without the negative integer using a different order
    # (namely 5 = 6 + 2 - 3).
    if guess - el <= 0 and op == operator.sub:
      return False
    # If a solution could use a fraction (e.g. 6 = 3 / 4 * 8) then it can be
    # accomplished without using the fraction (namely 6 = 8 * 4 / 3).
    if guess % el != 0 and op == operator.floordiv:
      return False
    # A solution should not require multiplication or division by 1, as it can
    # be accomplished without that operation.
    if el == 1 and (op == operator.mul or operator.floordiv):
      return False
    
    guess = operations[i](guess, elements[i+1])
    if guess == answer:
      return True
  
  return False

In [ ]:
def numbers(arr, m):
  """Solves the numbers question in Letters & Numbers.

  Uses any integers in arr (without repetition) and the four basic arithmetic
  operators (+, -, *, /) to arrive at the integer m.

  Args:
    arr: An array of positive integers, the numbers to work with.
    m: A positive integer, the target of the problem.
  
  Returns:
    Two lists, the integers and the operations that when combined in order (not
    using BODMAS) return m. Returns empty lists if no solutions found.
  """

  # Check that inputs are valid.
  if not _is_positive_integer(m):
    raise ValueError('Input m must be a positive integer.')

  if any(not _is_positive_integer(n) for n in arr):
    raise ValueError('All values in input arr must be positive integers.')

  for permutation_generator in all_permutations(arr):
    permutation = list(permutation_generator)

    if len(permutation) == 1 and permutation[0] == m:
      return permutation, []

    for operation in itertools.permutations(
        all_operations, len(permutation) - 1):
      if check_permutation(permutation, list(operation), m):
        return permutation, list(operation)
  
  return [], []

In [ ]:
numbers([2, 4, 6, 10, 25, 50], 170)

# Solving the Letters & Numbers game

The actual Letters and Numbers game is more specific. 

- A contestant chooses how many "small" numbers $N_S$ and how many "large" numbers $N_L$
- The small numbers are chosen from 1-10 inclusive **with replacement**
- The large numbers are chosen from 25, 50, 75, 100 **without replacement**

Are there any optimizations that can be made given these restrictions?

# Appendix: Recursive attempt

In [ ]:
def inner(arr, m, output_elements = [], output_operations = []):
  print(arr, m)
  # global output_elements, output_operations
  print(output_elements, output_operations)

  # If the product of all elements of arr exceeds m, no solution exists.
  p = np.prod(arr)
  if m > p:
    return [], []
  # Since np.prod(arr) is already calculated, may as well use it to test if the
  # product is equal to m.
  if m == p:
    return arr, ['*'] * (len(arr) - 1)

  # Base cases
  for n in arr:
    if n == m:
      return output_elements + [n], output_operations
  if len(arr) < 2:
    return [], []
  
  for i in range(len(arr)):
    n = arr[i]
    arr_minus_n = arr[:i] + arr[(i+1):]
    # Assume the last element of the expression is n.

    # Multiplication
    # Ignore the case where n == 1 since this is an identity operation.
    if n > 1 and m % n == 0:
      print('*')
      print(n)
      return inner(
          arr_minus_n, m // n, output_elements + [n], output_operations + ['*'])
    
    # Addition
    # Ignore the case where m - n <= 0 since this will raise an error in the
    # recursion. If a solution could use a negative integer (e.g. 5 = 2 - 3 + 6)
    # then it can be accomplished without the negative integer using a different
    # order (namely 5 = 6 + 2 - 3).
    if m - n > 0:
      print('+')
      print(n)
      return inner(
          arr_minus_n, m - n, output_elements + [n], output_operations + ['+'])
    
    # # Division
    # print('/')
    # print(n)
    # return inner(
    #     arr_minus_n, m * n, output_elements + [n], output_operations + ['/']
    # )

    # # Subtraction
    # print('/')
    # print(n)
    # return inner(
    #     arr_minus_n, m + n, output_elements + [n], output_operations + ['-']
    # )

In [ ]:
print(inner([5, 2], 3))

In [ ]:
def numbers(arr, m):
  """Solves the numbers question in Letters & Numbers.

  Uses any integers in arr (without repetition) and the four basic arithmetic
  operators (+, -, *, /) to arrive at the integer m.

  Args:
    arr: An array of positive integers.
    m: A positive integer.
    output_elements: Keeps track of the elements in the final arithmetic
      expression. Not to be entered manually.
    output_operations: Keeps track of the operations in the final arithmetic
      expression. Not to be entered manually.
  
  Returns:
    Something cool.
  """

  # Check that inputs are valid.
  if not _is_positive_integer(m):
    raise ValueError('Input m must be a positive integer.')

  if any(not _is_positive_integer(n) for n in arr):
    raise ValueError('All values in input arr must be positive integers.')

  output_elements, output_operations = [], []

  solution_found = inner(arr, m)
  if solution_found:
    return output_elements, output_operations
  # else:
  #   return numbers(arr[1:] + arr[0], m)

In [ ]:
print(numbers([2, 3], 5))